# 5-6-如何使用临时文件

## 实际案例
在某项目中，我们从传感器采集数据，每收集到1G数据后，做数据分析，最终只保存分析结果。这样很大的历史数据如果常驻内存，将消耗大量内存资源，我们可以使用临时文件存储这些临时数据（外部存储）。  
临时文件可以不用命名，并且关闭后会自动删除。

## TemporaryFile

In [1]:
from tempfile import TemporaryFile

In [2]:
tf = TemporaryFile()
tf.write(b"*" * 1024 * 1024)
tf.write(b"*" * 1024 * 1024)
tf.write(b"*" * 1024 * 1024)
tf.write(b"*" * 1024 * 1024)
tf.seek(0)
tf.read(512)

b'********************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************'

`特殊点`:
1. 在文件系统中是找不到的
2. `tf.close()`临时文件就会自动地被删除掉

### 验证tf.close()

In [3]:
!df -h # 查看磁盘使用量

Filesystem      Size  Used Avail Use% Mounted on
udev            1.9G     0  1.9G   0% /dev
tmpfs           394M  716K  393M   1% /run
/dev/vda2        79G  9.0G   67G  12% /
tmpfs           2.0G   24K  2.0G   1% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs           2.0G     0  2.0G   0% /sys/fs/cgroup
tmpfs           394M     0  394M   0% /run/user/1000


In [4]:
tf = TemporaryFile()
tf.write(b"*" * 1024 * 1024 * 1024) 

1073741824

In [5]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev            1.9G     0  1.9G   0% /dev
tmpfs           394M  716K  393M   1% /run
/dev/vda2        79G   10G   66G  14% /
tmpfs           2.0G   24K  2.0G   1% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs           2.0G     0  2.0G   0% /sys/fs/cgroup
tmpfs           394M     0  394M   0% /run/user/1000


In [6]:
tf.close()
!df -h

Filesystem      Size  Used Avail Use% Mounted on
udev            1.9G     0  1.9G   0% /dev
tmpfs           394M  716K  393M   1% /run
/dev/vda2        79G  9.0G   67G  12% /
tmpfs           2.0G   24K  2.0G   1% /dev/shm
tmpfs           5.0M     0  5.0M   0% /run/lock
tmpfs           2.0G     0  2.0G   0% /sys/fs/cgroup
tmpfs           394M     0  394M   0% /run/user/1000


### 此外
1. `Temporaryfile`创建的是一个操作系统级别的临时文件  
    系统级别的open,就是`os.open`  
    在`ubuntu`里，可以使用`man 2 open`查看linux命令open的c语言帮助，里面open函数可以接受参数`O_TMPFILE`参数，加上这个参数就可以创建系统级别的临时文件。  
2. `Tempporary`可以接受`dir`参数，指定位置(虽然看不到这个文件)
    用处，比如电脑磁盘有两个分区:  
    sda1 -> /
    sda2 -> /opt
    此时指定`/opt`作为`dir`参数的值就可以在sda2下写临时文件

## NamedTemporaryFile
1. 可以命名，可以在磁盘中找到这个文件。
2. 不是操作系统级别的，即临时文件是python库做的，而不是操作系统做的

In [7]:
from tempfile import NamedTemporaryFile

In [8]:
ntf = NamedTemporaryFile()

In [9]:
ntf.name

'/tmp/tmpgc6vf27a'

### 获取默认值

In [10]:
import tempfile

In [11]:
tempfile.gettempdir()

'/tmp'

In [12]:
tempfile.gettempprefix()

'tmp'

### other

In [13]:
ntf.close() # 关闭时自动删除
# 然而可以 ntf = NamedTemporaryFile(datele=False)来取消自动删除